In [2]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a word to replace the following: {word}?"

llm = OpenAI(model_name='text-davinci-003', temperature=0)

llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))


# The __call__ method is used when providing a single input
llm_chain('artificial')

{'word': 'artificial', 'text': '\n\nSynthetic'}

In [3]:
# The apply method can be used with multiple inputs.
# Note that apply method does not return the original words like __call__
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'}, {'text': '\n\nWisdom'}, {'text': '\n\nAutomaton'}]

In [4]:
# The generate method returns more information
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nWisdom', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nAutomaton', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 13, 'total_tokens': 46, 'prompt_tokens': 33}, 'model_name': 'text-davinci-003'}, run=RunInfo(run_id=UUID('ff658647-8c2e-411c-b742-f847fdd083f7')))

In [5]:
# 
prompt_template = "Looking at the context of '{context}'. What is an appropriate word to replace the following: {word}?"

llm_chain = LLMChain(llm=llm, 
                     prompt=PromptTemplate(template=prompt_template, 
                                           input_variables=['word', 'context']))

llm_chain.predict(word='fan', context='object')

'\n\nventilator'

In [7]:
llm_chain.predict(word='fan', context='humans')

'\n\nAdmirer'

Parsers

In [13]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""
prompt = PromptTemplate(template=template, output_parser=output_parser, input_variables=[])

llm_chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)

llm_chain.predict()

['Synthetic',
 'Manufactured',
 'Imitation',
 'Fabricated',
 'Fake',
 'Simulated',
 'Artificial Intelligence',
 'Automated',
 'Constructed',
 'Programmed',
 'Mechanical',
 'Processed',
 'Algorithmic',
 'Generated.']

Conversational Chain (Memory)

In [19]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())

conversation.predict(input="List all possible words as substitute for 'artificial' as comma separated.")

' Synthetic, robotic, manufactured, simulated, computerized, programmed, man-made, fabricated, contrived, and artificial.'

Sequential Chain

In [20]:
from langchain.chains import SimpleSequentialChain

# overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two])

Custom Chain

In [21]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from typing import Dict, List

class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)
    
    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']
    
    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}
    


prompt_1 = PromptTemplate(input_variables=['word'],
                          template="What is the meaning of the following word '{word}'?")
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(input_variables=['word'],
                          template="What is a word to replace the following: {word}")
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run('artificial')
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Artificial means something that is not natural or made by humans, but rather created or produced by artificial means.

Synthetic
